In [1]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string

In [2]:
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor
from slimit import minify

In [3]:
# connecting to my mongodb and get data out
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [4]:
# import data to dataframe
tomato_data = pd.DataFrame(list(col.find()))
goal_list = tomato_data.loc[tomato_data['cretic(%)'] == 'na'][['name', 'url', 'year']]

In [5]:
goal_list

,name,url,year
0,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_ep_...,2015
2,Black Panther,https://www.rottentomatoes.com/m/black_panther,2018
6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
7,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,2017
11,Beauty and the Beast,https://www.rottentomatoes.com/m/beauty_and_th...,2017
13,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_ep_...,1999
14,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars_ep_...,1977
20,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006
36,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_ep_...,2005
38,Beauty and the Beast,https://www.rottentomatoes.com/m/beauty_and_th...,1991


In [6]:
#search movie in rotten tomato
base_url = 'https://www.rottentomatoes.com/search/?search='
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        
        return r.text
    except:
        print(url + 'ERROR')
        pass

In [7]:
def get_movie_list(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    movie_list = {}
    try:
        movie_list = soup.find_all('script')[32]
    except:
        movie_list = 'na'
        
    return movie_list

In [8]:
def get_movie_url(data, movie, year):
    text = data.text
    parser = Parser()
    tree = parser.parse(text)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            if(field['name'] == movie):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                
                if year_dic['year'] == str(year):
                    url_s = url['url']
                
        except:
            pass
        
    return url_s

In [218]:
for index, row in goal_list.iterrows():
    url_try = base_url + row['name']
    data = get_movie_list(url_try)
    url_find = get_movie_url(data, row['name'], row['year'])
    print(url_find)

to find


/m/black_panther_2018


to find


to find


/m/beauty_and_the_beast_2017


...

/m/harry_potter_and_the_half_blood_prince


/m/sixth_sense


to find


/m/superman_man_of_steel


to find
/m/gravity_2013


In [163]:
text = new_data
parser = Parser()
tree = parser.parse(text)

In [149]:
fields = []
for node in nodevisitor.visit(tree):
    if isinstance(node, ast.Assign):
        try:
            field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
            fields.append(field)   
        except:
            pass

In [162]:
for index, field in enumerate(fields):
    #print(field)
    try:
        if(field['name'] == 'Black Panther'):
            year = fields[index + 1]
            print(year)
            url = fields[index + 2]
            if year['year'] == '2018':
                url_b = url['url']
                print(url_b)
                
            print('....')
    except:
        pass

{'year': '1977'}
....
{'year': '2018'}
/m/black_panther_2018
....
